# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调

这个 Notebook 会自动配置环境、启动微调 (QLoRA)、运行测试、合并 LoRA 适配器并导出 GGUF 模型。

**配置**：您可以通过修改下面的第一个代码单元格中的变量来指定要使用的基础模型和数据集。

**步骤：**
1. 配置变量、克隆/更新 GitHub 仓库并定义路径。
2. 安装必要的依赖项。
3. 运行训练脚本 (QLoRA)。
4. (可选) 运行测试脚本与微调后的模型(适配器)交互。
5. 合并 LoRA 适配器到基础模型。
6. (可选) 将合并后的模型转换为 GGUF 格式。

In [ ]:
# 1. 配置、克隆/更新仓库并定义路径
import os
import sys
# 尝试导入 rich 用于美化输出，如果失败则忽略
try:
    from rich import print
    rich_available = True
except ImportError:
    rich_available = False
    # 使用内置 print
    pass

# ==============================================================
#                  主要配置区域 (在此处修改)
# ==============================================================

# --- A. ⚠️核心配置 ---
# 指定要微调的基础模型 (Hugging Face 名称或路径)
base_model_name: str = "Qwen/Qwen2.5-7B-Instruct" # <--- 确认这里是正确的模型名称
# 指定数据集文件路径 (相对于仓库根目录)
dataset_file: str = "data/converted_dataset.json"

# --- B. 训练超参数 (按需修改） ---
num_train_epochs: int = 2          # 训练轮数
learning_rate: float = 2e-4        # 学习率
weight_decay: float = 0.01         # 权重衰减
max_grad_norm: float = 1.0           # 梯度裁剪范数
seed: int = 42                   # 随机种子

# --- C. LoRA 特定参数 (按需修改) ---
lora_r: int = 16                 # LoRA 秩
lora_alpha: int = 32               # LoRA alpha
lora_dropout: float = 0.05           # LoRA dropout

# --- D. 保存与日志 (按需修改) ---
save_steps: int = 25               # 每 N 步保存一次 checkpoint
logging_steps: int = 5             # 每 N 步记录一次日志

# --- E. 输出目录 (高级，通常自动生成) ---
# 默认会根据 base_model_name 自动生成。
# 如果要自定义，请将 None 替换为你的路径字符串(相对路径)，例如: "output/my_custom_run"
custom_output_dir: str | None = None

# --- F. ⚠️工作空间配置 (!!! 重要 !!!) ---
# 指定仓库应该位于哪个父目录下。
# 你要自己看你的环境是Linux 还是Windows 主要是取决于你的训练环境，比如说 /mnt/workspace, /workspace, /home/user 等。
# 对于 Colab，固定为 /content。
target_workspace_dir: str = "/mnt/workspace"  # <--- 修改这里来匹配你的环境!
if 'google.colab' in sys.modules:
    target_workspace_dir = "/content" # Colab 环境固定

# ------------------------------------------


# ==============================================================
#          仓库克隆/更新和路径动态确定 (通常无需修改)
# ==============================================================

# --- 仓库相关设置 ---
repo_url: str = "https://github.com/shuakami/echoheart_demo.git"
# 仓库目录名
repo_dir_name: str = "echoheart_demo"

# --- 计算准确的仓库路径 ---
# 直接将工作空间路径和仓库目录名拼接
repo_path: str = os.path.join(target_workspace_dir, repo_dir_name)
repo_path = os.path.abspath(repo_path) # 确保是绝对路径

if rich_available: print(f"目标仓库路径: [bold magenta]{repo_path}[/bold magenta]")
else: print(f"目标仓库路径: {repo_path}")


# --- 克隆或更新仓库 ---
# 保存当前工作目录，以便后续恢复
original_cwd = os.getcwd()

# 确保目标工作空间目录存在
os.makedirs(target_workspace_dir, exist_ok=True)

# 切换到目标工作空间目录
os.chdir(target_workspace_dir)
if rich_available: print(f"已切换到工作空间目录: [cyan]{os.getcwd()}[/cyan]")
else: print(f"已切换到工作空间目录: {os.getcwd()}")


if not os.path.exists(repo_path): # 检查的是拼接好的绝对路径
  if rich_available: print(f"\n仓库不存在于 [cyan]{repo_path}[/cyan], 开始克隆...")
  else: print(f"\n仓库不存在于 {repo_path}, 开始克隆...")
  # 直接克隆到目标 repo_path (它现在是绝对路径)
  clone_result = os.system(f"git clone --depth 1 {repo_url} \"{repo_path}\"")
  if clone_result != 0:
      # 克隆失败后切换回原始目录
      os.chdir(original_cwd)
      raise RuntimeError(f"Git 克隆失败，请检查仓库 URL 和目标路径: {repo_path}")
  os.chdir(repo_path) # 克隆成功后进入仓库目录
else:
  if rich_available: print(f"\n仓库已存在于 [cyan]{repo_path}[/cyan].")
  else: print(f"\n仓库已存在于 {repo_path}.")
  os.chdir(repo_path) # 如果已存在，也进入仓库目录
  if rich_available: print("\n尝试拉取最新更改...")
  else: print("\n尝试拉取最新更改...")
  # 使用 --no-edit 避免潜在的编辑器冲突，并确保拉取成功
  pull_result = os.system("git pull origin master --no-edit --ff-only")
  if pull_result != 0:
      if rich_available: print(f"[yellow]Git pull 失败 (可能由于本地更改或非快进合并). 将继续使用本地版本.[/yellow]")
      else: print(f"警告: Git pull 失败。将继续使用本地版本。")

# --- 确认最终的仓库路径 ---
final_repo_path = os.getcwd()
if final_repo_path != repo_path:
     print(f"[bold red]警告：最终工作目录 ({final_repo_path}) 与预期仓库路径 ({repo_path}) 不符！请检查逻辑。[/bold red]")
     repo_path = final_repo_path

if rich_available: print(f"\n[green]确认当前工作目录 (仓库路径):[/green] [bold magenta]{repo_path}[/bold magenta]")
else: print(f"\n确认当前工作目录 (仓库路径): {repo_path}")


# ==============================================================
#          基于动态路径的计算 (通常无需修改)
# ==============================================================

# --- 确定最终适配器输出目录 ---
# 如果用户没有设置 custom_output_dir，则自动生成
if custom_output_dir:
    # 用户自定义的是相对路径
    adapter_output_rel_dir: str = custom_output_dir
    if rich_available: print(f"使用自定义适配器相对输出目录: [blue]{adapter_output_rel_dir}[/blue]")
    else: print(f"使用自定义适配器相对输出目录: {adapter_output_rel_dir}")
else:
    # 自动生成相对路径
    adapter_output_rel_dir: str = f"output/{base_model_name.split('/')[-1]}-qlora-ft"
    if rich_available: print(f"使用自动生成的适配器相对输出目录: [blue]{adapter_output_rel_dir}[/blue]")
    else: print(f"使用自动生成的适配器相对输出目录: {adapter_output_rel_dir}")

# --- 计算绝对路径 (!!! 基于正确的 repo_path !!!) ---
adapter_path: str = os.path.join(repo_path, adapter_output_rel_dir)
dataset_abs_path: str = os.path.join(repo_path, dataset_file)

# --- 合并模型和 GGUF 的输出路径 (保持相对路径结构) ---
# 合并模型的目录名，基于适配器目录名
merged_model_output_rel_dir: str = adapter_output_rel_dir.replace('-qlora-ft', '-merged-ft')
# 合并模型的绝对路径
merged_model_path: str = os.path.join(repo_path, merged_model_output_rel_dir)
# GGUF 文件名
gguf_output_filename: str = f"gguf-model-f16.gguf"
# GGUF 的绝对输出路径 (放在合并后的模型目录里)
gguf_output_abs_path: str = os.path.join(merged_model_path, gguf_output_filename)

# ==============================================================
#          路径验证
# ==============================================================
print("\n--- 路径验证 ---")
print(f"仓库根目录 (repo_path): {repo_path}")
print(f"适配器目录 (adapter_path): {adapter_path}")
print(f"数据集文件 (dataset_abs_path): {dataset_abs_path}")
print(f"合并模型目录 (merged_model_path): {merged_model_path}")
print(f"GGUF 输出路径 (gguf_output_abs_path): {gguf_output_abs_path}")
print("------------------")

# ==============================================================
#          最终配置打印与检查
# ==============================================================

print("\n--- 最终配置信息 ---")
print(f"基础模型: {base_model_name}")
print(f"数据集文件 (绝对路径): {dataset_abs_path}")
print(f"适配器输出目录 (绝对路径): {adapter_path}")
print(f"学习率: {learning_rate}, Epochs: {num_train_epochs}, LoRA r: {lora_r}")
print(f"合并模型输出目录 (绝对路径): {merged_model_path}")
print(f"GGUF 输出文件 (绝对路径): {gguf_output_abs_path}")
print("----------------------")

# 确保数据集文件存在
if not os.path.exists(dataset_abs_path):
  err_msg = f"错误：指定的数据集文件不存在: {dataset_abs_path}"
  if rich_available: print(f"\n[bold red]{err_msg}[/bold red]")
  else: print(f"\n{err_msg}")
  # 引发错误停止执行
  raise FileNotFoundError(err_msg)
else:
  ok_msg = f"数据集文件确认存在: {dataset_abs_path}"
  if rich_available: print(f"\n[green]{ok_msg}[/green]")
  else: print(f"\n{ok_msg}")

print("\n最终工作目录 (应为仓库根目录):")
# 使用 os.system 执行 pwd/cd，确保显示的是 shell 的当前目录
if os.name == 'nt': # Windows系统
    os.system("cd")
else: # Linux/macOS系统
    os.system("pwd")

In [ ]:
# 2. 安装依赖
# 安装 ModelScope (如果需要)
try:
    # 检查是否可以连接到 Hugging Face
    import socket
    can_connect_to_hf = False
    try:
        socket.setdefaulttimeout(2.0)
        socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(("huggingface.co", 443))
        can_connect_to_hf = True
        print("已检测到 Hugging Face 连接正常，不需要使用备选下载源。")
    except:
        print("无法连接到 Hugging Face，将尝试设置 ModelScope 作为备选下载源。")
        
    if not can_connect_to_hf:
        print("正在安装 ModelScope 以便从国内镜像下载模型...")
        # 安装 ModelScope
        !pip install -q modelscope
        
        # 设置 ModelScope 环境变量
        import os
        os.environ["MODELSCOPE_CACHE"] = os.path.expanduser("~/.cache/modelscope/hub")
        os.environ["VLLM_USE_MODELSCOPE"] = "True"
        os.environ["USE_MODELSCOPE_CACHE"] = "1"
        
        print(f"ModelScope 已设置为模型下载源，模型将缓存在: {os.environ['MODELSCOPE_CACHE']}")
        
        # 如果需要登录？
        # from modelscope import HubApi
        # api = HubApi()
        # api.login("YOUR_MODELSCOPE_TOKEN") # 替换为您的 token
        # print("已登录 ModelScope")
except Exception as e:
    print(f"ModelScope 设置失败: {e}")
    print("将继续使用 Hugging Face 下载模型。")

# 安装依赖
print("Installing dependencies from requirements.txt...")
!pip install -q -r requirements.txt

print("Applying dependency fixes...")
!pip install fsspec==2024.12.0

In [ ]:
# 3. 运行训练脚本 (QLoRA)
print("Starting QLoRA training...")

train_command = (
    f"python train.py "
    f"--base_model_name \"{base_model_name}\" "
    f"--dataset_file \"{dataset_file}\" "
    f"--output_dir \"{adapter_path}\" " 
    f"--num_train_epochs {num_train_epochs} "
    f"--learning_rate {learning_rate} "
    f"--weight_decay {weight_decay} "
    f"--max_grad_norm {max_grad_norm} "
    f"--lora_r {lora_r} "
    f"--lora_alpha {lora_alpha} "
    f"--lora_dropout {lora_dropout} "
    f"--save_steps {save_steps} "
    f"--logging_steps {logging_steps} "
    f"--seed {seed}"
)

print("\n--- Running Training Command ---")
print(train_command)
print("------------------------------\n")

# 执行命令
!{train_command}

## 训练完成！

微调后的模型保存在 Colab 环境文件系统的 `output/qwen-ft` 目录中。

In [ ]:
# 4. 运行测试脚本
print("Starting non-interactive testing session...")
!python test_model.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}"

## 5. (可选) 转换为 GGUF 格式

转换后的文件将尝试保存在 `{gguf_output_file}`。

In [ ]:
# 运行 LoRA 合并脚本
print("Starting LoRA merge...")
!python merge_lora.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}" --output_path "{merged_model_path}"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir "{merged_model_path}" --output_file "{gguf_output_abs_path}" --out_type f16

## (可选) 查看 train.py 的高级参数

运行下面的代码单元格可以显示 `train.py` 脚本支持的所有命令行参数及其说明和默认值。
如果您想覆盖默认设置（例如调整学习率、LoRA rank、保存步数等），可以在第 3 步运行训练时手动添加这些参数。

In [ ]:
!python train.py --help